# Overview

Script of Scripts (SoS) is a scripting language designed for the execution of workflows that involve the analysis of data in multiple languages. It allows the integration of scripts written in various languages, providing a unified and flexible environment for data analysis.

